In [ ]:
import numpy as np
from numpy import random
import json

import string
alphabet_str = string.ascii_lowercase
print(alphabet_str)
N = len(alphabet_str)
az_list = list(alphabet_str)
print(az_list)

In [ ]:
# As for the encoding schema, 
# let's start with a general fixed letter to fixed letter permutation first.

# Choose one of the 26! letter permutations randomly. 
# With a fixed seed parameter the same one can be chosen repeatedly.
def construct_random_letter_permutation(seed=None):
    if seed != None:
        random.seed(seed)
    letter_indices = np.array(list(range(0,len(alphabet_str))))
    #print(letter_indices)
    letter_indices_permutated = letter_indices.copy()
    letter_indices_permutated = random.permutation(letter_indices_permutated)
    #print(letter_indices_permutated)    
    letter_permutation_dict = {}
    for letter_idx_in in letter_indices:
        letter_idx_out = letter_indices_permutated[letter_idx_in]
        letter_permutation_dict[alphabet_str[letter_idx_in]] = alphabet_str[letter_idx_out] 
    return letter_permutation_dict

# enoode a message with a letter permutation
def encode(msg, letter_permutation):
    enc_msg = ''
    for i in range(len(msg)):
        enc_msg += letter_permutation[msg[i]]
    return enc_msg

In [ ]:
plaintext_msg = 'Let us try to decrypt with a quantum computer.'
# Remove spaces and dots, and restrict to lowercase characters only, so we have 26 symbols.
plaintext_msg = plaintext_msg.replace(' ', '').replace('.', '').lower()

print('plaintext message = {:s}'.format(plaintext_msg))
letter_permutation_dict = construct_random_letter_permutation(3)
print('letter permutation dictionary =')
print(json.dumps(letter_permutation_dict, indent=2))
enc_msg = encode(plaintext_msg, letter_permutation_dict)
print('encoded message = {:s}'.format(enc_msg))

In [ ]:
inverse_letter_permutation_dict = {v: k for k, v in letter_permutation_dict.items()}
print('inverse letter permutation dictionary =')
print(json.dumps(inverse_letter_permutation_dict, indent=2))
recovered_msg = encode(enc_msg, inverse_letter_permutation_dict)
print('recovered message = {:s}'.format(recovered_msg))

In [ ]:
# From view-source:http://norvig.com/mayzner.html, we get the unigrams data

In [ ]:
unigram = { 'e': 12.49, 't': 9.28, 'a': 8.04, 'o':7.64, 'i': 7.57, 'n':7.23, 's': 6.51, 'r': 6.28, 'h': 5.05, 'l':4.07, 'd':3.82, 'c':3.34, 'u':2.73, 'm': 2.51, 'f':2.40, 'p': 2.14, 'g': 1.87, 'w': 1.68, 'y': 1.66, 'b': 1.48, 'v':1.05, 'k': 0.54, 'x': 0.23, 'j': 0.16, 'q': 0.12, 'z': 0.09 }

In [ ]:
total = 0.0
for k, v in unigram.items():
    total += v

In [ ]:
total, len(unigram.keys())

In [ ]:
# define the permutation vars. 26 ints or 26*26 bools? 
# say: permutation as ints. p[f] = t or 1. f/t = from/to, f in [0,25], t in [0, 25].
# constraints: avoiding the ones we'd have to impose for bool vars:
# say p is the decode permutation, mapping encoded letter to decoded one. p is to be found var.
# sum_f p[f][t] = 1 for all t
# sum_t p[f][t] = 1 for all f
# objective: achieve typical fraction for all letters.
# Minimize quadratic errors = energy function?
#sum_{l in enc_text} sum_o (f[p[ord(l)-ord('a')][]] - f[])^2

In [ ]:
import gurobipy as gp
from gurobipy import GRB
m = gp.Model()

print('ord(a) = {:d}'.format(ord('a')))
#p = m.addMVar((N,N), vtype=GRB.BINARY) # p[a][b] is the encoding permutation -> p[b][a] encodes
p = m.addVars(az_list, az_list, vtype=GRB.BINARY, name='p') # p[a][b] is the encoding permutation -> p[b][a] encodes
for y in az_list:
    row_sum = 0
    for x in az_list:
        row_sum += p[y, x]
    m.addConstr(row_sum == 1, 'row_{:s}_sum_is_1'.format(y))
for x in az_list:
    col_sum = 0
    for y in az_list:
        col_sum += p[y, x]
    m.addConstr(col_sum == 1, 'col_{:s}_sum_is_1'.format(x))

# relation between original letter and encrypted letter
for msg_let_idx, enc_let in enumerate(enc_msg):
    enc_let_ord = ord(enc_let) - ord('a')
    assert enc_let_ord <= N-1
    assert enc_let_ord >= 0
    print('msg_let_idx:{:d}, enc_let:{:s}, enc_let_ord:{:d}'.format(msg_let_idx, enc_let, enc_let_ord))
    expr = 0
    for x in az_list:
        bit = p[enc_let, x]
        expr += bit * (ord(x) - ord('a'))
    m.addConstr(expr <= N-1, 'le_msg_let_idx_{:d}_enc_let_{:s}_enc_let_ord_{:d}'.format(msg_let_idx, enc_let, enc_let_ord))
    m.addConstr(expr >= 0, 'ge_msg_let_idx_{:d}_enc_let_{:s}_enc_let_ord_{:d}'.format(msg_let_idx, enc_let, enc_let_ord))

#p_dec = m.addVars(permutation_dec, lb=[0]*N, ub=[25]*N, vtype=GRB.BOOL, name='p')  
m.write('decode.lp')
m.optimize()


In [ ]:
#print(p.x)

In [ ]:
# two letters that are the same in the encrypted message originate from two letters in the 
# original plain text message that are also the same.
# This is already embedded (hard coded) in the way the permutation constraitn was formulated.

# unigram statistics

# bigram statistics


In [ ]:
# Reconstruct the message from the permutation matrix found.
def codec(enc_msg, decode_io_code=True):
    dec_msg = ''
    for msg_let_idx, enc_let in enumerate(enc_msg):
        enc_let_ord = ord(enc_let)-ord('a')
        assert enc_let_ord < 26
        assert enc_let_ord >= 0
        print('msg_let_idx:{:d}, enc_let:{:s}, enc_let_ord:{:d}'.format(msg_let_idx, enc_let, enc_let_ord))
        
        dec_let_ord = 0
        for x in az_list:
            if decode_io_code:
                bit = int(p[x, enc_let].x)  # decode. 
                # The inverse matrix of a permutation matrix is its transposed.
            else:
                bit = int(p[enc_let, x].x)  # encode
                
            dec_let_ord += bit * (ord(x) - ord('a'))
            
        dec_let = chr(ord('a') + dec_let_ord)
        dec_msg += dec_let
    return dec_msg
    
    
print('enc_msg: {:s}'.format(enc_msg))
    
dec_msg = codec(enc_msg, decode_io_code=True)
print('dec_msg: {:s}'.format(dec_msg))
assert len(enc_msg) == len(dec_msg)

enc_again_msg = codec(dec_msg, decode_io_code=False)
print('enc_again_msg: {:s}'.format(enc_again_msg))
assert len(enc_again_msg) == len(dec_msg)

